In [23]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

In [24]:
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df_y = pd.DataFrame(iris.target)

In [25]:
df=pd.concat([df, df_y], axis=1)

In [26]:
### lets rename the columns 
df.columns= ['sepal_len','sepal_wid','petal_len','petal_wid','target']

In [27]:
df.head()

,sepal_len,sepal_wid,petal_len,petal_wid,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [28]:
df['target']= df['target'].apply(int)

In [29]:
y= df['target']
X = df.drop('target',axis=1)

##  LETS TRAIN TEST SPLIT FOR TENSORFLOW

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Lets create tensor flow feature columns  - tf object is already created at the top

In [31]:
##  please note that columns are numerical and not categorical. For categorical you will use a different function
feat_col=[]
for col in X.columns:
    feat_col.append(tf.feature_column.numeric_column(col)) ###

In [32]:
len(feat_col)

4

In [33]:
feat_col

[_NumericColumn(key='sepal_len', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_wid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_len', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_wid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Lets create input function , For training 

In [66]:
my_inputFunc = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=5,num_epochs=8,shuffle=True) 

## Build the model and train the model 

In [67]:
myClassifer = tf.estimator.DNNClassifier(hidden_units=[25,15,10,5],n_classes=3,feature_columns=feat_col)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\obaid\\AppData\\Local\\Temp\\tmpz9jdgom_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000235D04218D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [68]:
myClassifer.train(input_fn=my_inputFunc,steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\obaid\AppData\Local\Temp\tmpz9jdgom_\model.ckpt.
INFO:tensorflow:loss = 5.4912553, step = 1
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\obaid\AppData\Local\Temp\tmpz9jdgom_\model.ckpt.
INFO:tensorflow:Loss for final step: 0.99965876.


## Now lets test the model. So create a predicting input function 

In [69]:
pred_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size= len(X_test),shuffle=False) 

In [70]:
predictions = list(myClassifer.predict(input_fn=pred_func))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\obaid\AppData\Local\Temp\tmpz9jdgom_\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [71]:
predictions  ### they come in list and dictionary format. So lets get them out in a single list format

[{'logits': array([ 4.2902184, -0.6548536, -4.394448 ], dtype=float32),
  'probabilities': array([9.9276525e-01, 7.0669013e-03, 1.6793614e-04], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([ 3.989285  , -0.79370356, -4.2595224 ], dtype=float32),
  'probabilities': array([9.9144137e-01, 8.2993004e-03, 2.5933149e-04], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-1.2775803,  3.5995965,  3.9431617], dtype=float32),
  'probabilities': array([0.00315128, 0.4136361 , 0.5832126 ], dtype=float32),
  'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object)},
 {'logits': array([-1.3052151,  3.7153888,  4.056473 ], dtype=float32),
  'probabilities': array([0.00273533, 0.41440955, 0.58285517], dtype=float32),
  'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object)},
 {'logits': array([ 3.5079284, -0.2539435, 

In [72]:
final_preds = []
for item in predictions:
    final_preds.append(item['class_ids'][0])

In [73]:
final_preds

[0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0]

In [74]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test,final_preds))

[[18  0  0]
 [ 0  2 13]
 [ 0  0 12]]


In [75]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.13      0.24        15
           2       0.48      1.00      0.65        12

   micro avg       0.71      0.71      0.71        45
   macro avg       0.83      0.71      0.63        45
weighted avg       0.86      0.71      0.65        45

